In [2]:
# !pip install pymysql
# !pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile

import pandas as pd
import numpy as np

#jobName = '201216_Train_pilot'
jobName = '201216_Train_split1'
x, y = LibData.GetXY_s_o(jobName)

#l_x, l_y = LibData.GetSplited5(x,y)


In [6]:
##### 알파 찾기 #######################################

from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
import numpy as np

tuned_parameters = {
    'alpha' : [1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05]
    # 너무 작은 알파에서는 최적해를 찾지 못함 --> 1에서부터 시작
    #'alpha' : [1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05]
    #rg = [np.float_power(10, i) for i in np.array([range(-2, 6)])]
}

gridSearcher = GridSearchCV(linear_model.Lasso(),
                            param_grid = tuned_parameters, 
                            cv=5, 
                            scoring='neg_mean_absolute_error',
                            verbose=2)

gridSearcher.fit(x, y['pv_claim'])

#'Temp','Temp',
LibUtils.WriteNote('T_ParamSearch_Lasso','grid_cv5_data10_negmae',
                   [gridSearcher.param_grid, gridSearcher.best_params_, gridSearcher.best_score_])

LibUtils.ELoop()

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] alpha=1.0 .......................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ........................................ alpha=1.0, total=   4.3s
[CV] alpha=1.0 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s remaining:    0.0s


[CV] ........................................ alpha=1.0, total=   4.9s
[CV] alpha=1.0 .......................................................
[CV] ........................................ alpha=1.0, total=   3.6s
[CV] alpha=1.0 .......................................................
[CV] ........................................ alpha=1.0, total=   4.6s
[CV] alpha=1.0 .......................................................
[CV] ........................................ alpha=1.0, total=   3.9s
[CV] alpha=10.0 ......................................................
[CV] ....................................... alpha=10.0, total=   0.7s
[CV] alpha=10.0 ......................................................
[CV] ....................................... alpha=10.0, total=   0.6s
[CV] alpha=10.0 ......................................................
[CV] ....................................... alpha=10.0, total=   0.6s
[CV] alpha=10.0 ......................................................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   29.0s finished


start empty loop.

KeyboardInterrupt: ignored

In [16]:
##### 학습 #####################################

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import linear_model

# 3차원 하면 램이 뻗음, 2차원까지만 할 수 있음
x_poly = pd.DataFrame(PolynomialFeatures(2).fit_transform(x))

xTrain, xVal, yTrain, yVal = train_test_split(x_poly, y, test_size=0.2)

model = linear_model.Lasso(alpha=0.1)
model.fit(xTrain, yTrain['pv_claim'])

yPred = model.predict(xTrain, yTrain['pv_claim'])
LibUtils.GetScores(yVal['pv_claim'], yPred)

Lasso(alpha=0.1)

In [23]:
##### 중요변수 추출 #####################################

from sklearn.model_selection import train_test_split

xTrain, xVal, yTrain, yVal = train_test_split(x, y, test_size=0.2)

model = linear_model.Lasso(alpha=0.1)
model.fit(xTrain, yTrain['pv_claim'])

importance = pd.concat([pd.Series(xVal.columns), pd.Series(model.coef_)], axis=1)
importance.columns=['name','coeff']

# 베타가 0 이상인 것 고름, 16개 골라짐
picked = np.array(importance.loc[importance['coeff'] > 0.0]['name'])
picked, len(picked)

(array(['ContAge', 'PremYr', 'AccumPrem', 'ContAmt', 'FundVal03',
        'FundAllo03', 'IR10', 'IR11', 'prd_4', 'prd_5', 'prd_6', 'prd_7',
        'prd_8', 'prd_9', 'prd_10', 'prd_11'], dtype=object),
 16)